In [9]:
from random import randint
import os
from sys import getsizeof

In [7]:
def create_data(filename='data.txt', size=17179869184):
    length = 0
    with open(filename, 'wt') as file:
        while length < size:
            n = randint(0, 4294967295)
            s = f'{n}\n'
            file.write(s)
            length += getsizeof(s)

In [10]:
def read_data(multi=False):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-10-5a31d04bf139>, line 1)

In [11]:
print(0)
f = os.fork()
if f:
    print(0)
    ff = os.fork()
    if ff:
        print(0)
    else:
        print(2)
else:
    print(1)

0


AttributeError: module 'os' has no attribute 'fork'